In [137]:
%%capture
%run 01_transaction.ipynb

In [138]:
from eth_account.messages import encode_defunct as encode_msg
import web3 as w3; w3 = w3.Account

Create a public key `pub_key`, private key `priv_key` pair. The public key is used as an address and the private key is used for signing txs.

In [139]:
def keys():
    acc = w3.create()
    return acc.privateKey.hex(), acc.address

priv_key,pub_key = keys()
ph(priv_key), ph(pub_key)

('💂 0x1ca9759df39045...6dc', '🔆 0xA06a941aE6d01b...96D')

## Account

Holds a key pair and is used for signing txs. The `nonce` keeps track of the number of txs made with this account.

In [140]:
class Account:
    def __init__(self): 
        self.priv, self.pub = keys()
        self.nonce = 0
        
    def sign(self, tx):
        self.nonce += 1
        m = encode_msg(bytes(tx))
        sig = w3.sign_message(m, self.priv)
        tx.sig = sig
        return tx
            
    def signed_tx(self, to, value, fee):
        tx = TX(self.pub, to.pub, value, fee, self.nonce)
        return self.sign(tx)
    
    def __str__(self): return ph(self.pub)

In [141]:
acc1 = Account(); print(acc1)
acc2 = Account(); print(acc2)

💑 0x2b8Af3C2689d87...A8F
👫 0x058750B000A1E1...B9A


### Transaction Signing
Signing a tx generates a signature that only the private key can produce. This can be later validated very efficiently.

In [164]:
tx        = TX(acc1.pub, acc2.pub, 12, 0.2, acc1.nonce)
tx_signed = acc1.sign(tx)

The signature contains the tx hash `message_hash` and its `signature`. We don't need `r`, `v` and `s` as they are specific to Ethereum. For more infos: https://medium.com/@angellopozo/ethereum-signing-and-validating-13a2d7cb0ee3

In [165]:
tx_signed.sig

SignedMessage(messageHash=HexBytes('0x71d8fd7082001e23b05b4f47bb253c27ae4c7229ba45865d7dfbb24761649284'), r=41253418681712160653660556631771698333316511246469996054951677087836684145680, s=46713086180180228506975674296644322416030768412247138182178000942827167524712, v=27, signature=HexBytes('0x5b349b871c640590bad1c6d7610d557d65b9be6762d28d80a538d5578987d4106746ab79c56acc56d21133c0efc35c40b742b61e8f6b0af433f69ed159afa7681b'))

Accounts can generate a signed tx easily with `signed_tx`.

In [166]:
tx = acc1.signed_tx(acc2, 7, 0.3); print(tx)

time:	Sat Mar 27 10:44:36 2021
from:	💑 0x2b8Af3C2689d87...A8F
to:	👫 0x058750B000A1E1...B9A
value:	7 ether
fee:	0.3 ether
nonce:	6
hash:	🔬 c67d1478f9127d61...b66
signed:	true


### Transaction Validation

Validates if the tx was signed by the sender.

In [167]:
def val_sig(tx):
    if not hasattr(tx, 'sig'): return False
    m = encode_msg(bytes(tx))
    return w3.recover_message(m, signature=tx.sig.signature) == tx.fr

In [168]:
assert val_sig(tx_signed)

If anything in the tx is changed, like increasing the value to send, the signature should become invalid.

In [169]:
tx_signed.value = 30
assert not val_sig(tx_signed)